In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

In [0]:
import eli5
from eli5.sklearn import PermutationImportance

In [0]:
%pwd

'/content'

In [0]:
%ls

drive/  sample_data/


In [0]:
cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [0]:
%ls

'ColabDriveGithub (1).ipynb'   DAY4_meta.ipynb   dw_matrix/
 ColabDriveGithub.ipynb        DAY5.ipynb


In [0]:
cd dw_matrix/

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
ls


data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [0]:
df = pd.read_csv('data/men_shoes.csv')
df.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(18280, 48)

In [0]:
def run_model(features, model = DecisionTreeRegressor(max_depth=5)):
  X = df[features].values
  y = df.prices_amountmin.values
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df.brand.map(lambda x: str(x).lower()).factorize()[0] # assign ids
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [0]:
from ast import literal_eval # parse dictionary string to dict
from tqdm import tqdm_notebook

In [0]:
def parse_features0(x):
  if str(x) == 'nan' : return []
  return literal_eval(x.replace('\\"','"'))
df["features_parsed"] = df.features.map(parse_features0) 
df.features_parsed.head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [0]:
l = [
{'key': 'Gender', 'value': ['Men']}, 
{'key': 'Shoe Size', 'value': ['M']}, 
{'key': 'Shoe Category', 'value': ["Men's Shoes"]}, 
{'key': 'Color', 'value': ['Multicolor']}, 
{'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, 
{'key': 'Brand', 'value': ['Josmo']}
]

In [0]:
def parse_features(x):
  out = {}
  if str(x) == 'nan' : return out
  fs = literal_eval(x.replace('\\"','"'))
  for item in fs:
    key = item["key"].lower().strip()
    value = item["value"][0].lower().strip()
    out[key]=value
  return out
df["features_parsed"] = df.features.map(parse_features) 
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [0]:
keys = set()
df.features_parsed.map(lambda x: keys.update(x.keys()))
len(keys )

476

In [0]:
def get_name_key(k):
  return "feat_" + k
for key in tqdm_notebook(keys) : # tqdm_notebook for tracking progress
  df[get_name_key(key)] = df.features_parsed.map(lambda fs: fs[key] if key in fs else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_sole', 'feat_item weight', 'feat_hood', 'feat_location - country',
       'feat_is weather-resistant', 'feat_disclosures', 'feat_alarm',
       'feat_date first available', 'feat_footwear type', 'feat_hunting'],
      dtype='object', length=526)

In [0]:
df[ df['feat_athlete'].isnull() ].shape, df.shape

((18272, 526), (18280, 526))

In [0]:
df[ df['feat_athlete'].notnull() ].feat_athlete

4789    halloween costumes
4790    halloween costumes
4791    halloween costumes
4792    halloween costumes
4793    halloween costumes
4794    halloween costumes
4795    halloween costumes
4796    halloween costumes
Name: feat_athlete, dtype: object

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_key(key)].isnull()].shape[0]/df.shape[0]

In [0]:
{k:v for k,v in keys_stat.items() if v > 0.30}

{'brand': 0.4862691466083151,
 'color': 0.4778446389496718,
 'gender': 0.5017505470459519,
 'manufacturer part number': 0.362527352297593,
 'material': 0.34907002188183806}

In [0]:
df['feat_brand_cat'] = df.feat_brand.factorize()[0] # assign ids
df['feat_color_cat'] = df.feat_color.factorize()[0] 
df['feat_gender_cat'] = df.feat_gender.factorize()[0] 
df['feat_manufacturer part number_cat'] = df["feat_manufacturer part number"].factorize()[0] 
df['feat_material_cat'] = df.feat_material.factorize()[0] 

In [0]:
df.brand = df.brand.map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand].shape

(8846, 531)

In [0]:
#feats = ['feat_brand_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.318245383041685, 4.1738151265351515)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.153273340327715, 4.331194299935845)

In [0]:
X = df[feats].values
y = df.prices_amountmin.values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)
perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [0]:
feats2 = ['brand_cat', 'feat_brand_cat', 'feat_color_cat',  'feat_gender_cat', 'feat_material_cat']
run_model(feats2, model)

(-57.10887116193741, 4.307277645219528)

In [0]:
df.brand.value_counts(normalize=True)

nike                  0.097210
puma                  0.033315
ralph lauren          0.028775
vans                  0.021116
new balance           0.020295
                        ...   
cam newton nfl        0.000055
deadman wonderland    0.000055
peterluckyi           0.000055
g.h. bass & co.       0.000055
joseph abboud         0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
df[df.brand == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df['feat_sport_cat'] = df.feat_sport.factorize()[0] 
df['feat_style_cat'] = df.feat_style.factorize()[0] 
feats3 =  feats2 + ['feat_style_cat','feat_sport_cat','feat_fabric content_cat']

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats3, model)

In [0]:
print(result)
X = df[feats3].values
y = df.prices_amountmin.values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)
perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats3)

(-57.00948482862982, 4.212778445984782)


Weight,Feature
0.2534 ± 0.0079,brand_cat
0.1056 ± 0.0036,feat_material_cat
0.0450 ± 0.0016,feat_gender_cat
0.0255 ± 0.0007,feat_brand_cat
0.0168 ± 0.0015,feat_fabric content_cat
0.0096 ± 0.0013,feat_color_cat
0.0075 ± 0.0013,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [0]:
df[df.brand == 'nike'].features_parsed.sample(5).values

array([{'main color': 'black', 'country/region of manufacture': 'china'},
       {'sport': 'running shoes', 'material': 'suede', 'gender': 'men', 'shoe size': '8 men us', 'size': '8', 'shoe category': 'running shoes;training shoes;lifestyle;casual shoes', 'assembled product dimensions (l x w x h)': '13.00 x 8.00 x 4.00 inches', 'color': 'blue', 'model': '684773 404', 'casual & dress shoe style': 'running shoes;training shoes;lifestyle;casual shoes', 'manufacturer part number': '684773 404', 'variant group id': '224#mp#684773 404', 'brand': 'nike', 'age group': 'men', 'fabric content': 'suede', 'occasion': 'running shoes', 'shoe width': 'd(m)', 'shoe closure': 'lace-up'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'},
       {'style': 'running, cross training', 'country/region of manufacture': 'vietnam', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'dark pewter/black/grey/volt', 'casual & dress shoe style': 'a

In [0]:
for key in keys:
  df[ get_name_key(key) + '_cat'] = df[ get_name_key(key)].factorize()[0]
feats_cat = list(set([col for col in df.columns if 'cat' in col]))
print(feats_cat[1:11])

['feat_expandable_cat', 'feat_sizearm_cat', 'feat_eye size:_cat', 'feat_primary shelf id_cat', 'feat_foot arch_cat', 'feat_neckline_cat', 'feat_rx-able_cat', 'feat_shipping weight_cat', 'feat_fits most screen size_cat', 'feat_contained battery type_cat']


In [0]:
feats4 =  feats3 + ['feat_metal type_cat','feat_shape_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats4, model)

In [0]:
print(result)
X = df[feats4].values
y = df.prices_amountmin.values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)
perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats4)

(-57.24352793195519, 4.227672756816605)


Weight,Feature
0.2560 ± 0.0072,brand_cat
0.0984 ± 0.0100,feat_material_cat
0.0245 ± 0.0015,feat_gender_cat
0.0221 ± 0.0022,feat_brand_cat
0.0143 ± 0.0008,feat_fabric content_cat
0.0121 ± 0.0005,feat_shape_cat
0.0085 ± 0.0010,feat_metal type_cat
0.0076 ± 0.0008,feat_color_cat
0.0036 ± 0.0004,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [0]:
# TODO normalize weight column

In [148]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   HelloGithub.ipynb
	modified:   matrix_one/DAY4.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	matrix_one/DAY5.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!git add matrix_one/DAY5.ipynb

In [150]:
!git commit -m "work done on 5th day"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@236b5157c2dc.(none)')


In [0]:

!git config --global user.email "gotchas.u@gmail.com"